In [12]:
import pandas as pd

url = r"sqlite:///C:/Users/blinklet/Documents/Chinook_Sqlite.sqlite"


In [20]:
albums = pd.read_sql_table('Album', url)
print(df.shape)
display(df.head(3))

(347, 3)


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2


In [ ]:
statement = 'SELECT "Album"."Title", "Album"."ArtistId" FROM "Album"'
df = pd.read_sql_query(statement, url)
display(df.head(3))

In [ ]:
statement = 'SELECT "Album"."Title" AS "Album", \
    "Artist"."Name" AS "Artist", \
    "Track"."Name" AS "Track", \
    "Track"."Composer", \
    "Track"."Milliseconds" AS "Length" \
    FROM "Album" \
    JOIN "Track" ON "Album"."AlbumId" = "Track"."AlbumId" \
    JOIN "Artist" ON "Artist"."ArtistId" = "Album"."ArtistId"'

df = pd.read_sql_query(statement, url)
display(df.head(3))

In [21]:
artists = pd.read_sql_table('Artist', url)
tracks = pd.read_sql_table('Track', url)

In [17]:
statement = """
SELECT name from sqlite_master where type= "table";
"""
tables = pd.read_sql_query(statement, url)
print(tables)

             name
0           Album
1          Artist
2        Customer
3        Employee
4           Genre
5         Invoice
6     InvoiceLine
7       MediaType
8        Playlist
9   PlaylistTrack
10          Track


In [36]:
print(f"albums columns:  {albums.columns}\n")
print(f"artists columns:  {artists.columns}\n")
print(f"tracks columns:  {tracks.columns}")

albums columns:  Index(['AlbumId', 'Title', 'ArtistId'], dtype='object')

artists columns:  Index(['ArtistId', 'Name'], dtype='object')

tracks columns:  Index(['TrackId', 'Name', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer',
       'Milliseconds', 'Bytes', 'UnitPrice'],
      dtype='object')


In [53]:
df1 = (
    albums
    .merge(artists, on='ArtistId')
    .merge(tracks, on='AlbumId', suffixes=['_artist','_track'])
)
print(df1.shape)
display(df1.head(3))

(3503, 12)


,AlbumId,Title,ArtistId,Name_artist,TrackId,Name_track,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock We Salute You,1,AC/DC,1,For Those About To Rock (We Salute You),1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,1,For Those About To Rock We Salute You,1,AC/DC,6,Put The Finger On You,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99
2,1,For Those About To Rock We Salute You,1,AC/DC,7,Let's Get It Up,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99


In [55]:
df1 = (
    albums
    .merge(artists, on='ArtistId')
    .merge(tracks, on='AlbumId', suffixes=['_artist','_track'])
    .rename(columns = {'Title':'Album','Name_artist':'Artist', 
                       'Name_track':'Track', 'Milliseconds':'Length',})
    .drop(['AlbumId', 'TrackId', 'MediaTypeId', 
           'GenreId', 'Bytes', 'UnitPrice', 'ArtistId'], axis=1)
)

print(df1.shape)
display(df1.head(3))

(3503, 5)


,Album,Artist,Track,Composer,Length
0,For Those About To Rock We Salute You,AC/DC,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719
1,For Those About To Rock We Salute You,AC/DC,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662
2,For Those About To Rock We Salute You,AC/DC,Let's Get It Up,"Angus Young, Malcolm Young, Brian Johnson",233926


In [56]:
customers = pd.read_sql_table('Customer', url)
employees = pd.read_sql_table('Employee', url)

print(f"customers columns:  {customers.columns}\n")
print(f"employees columns:  {employees.columns}")

customers columns:  Index(['CustomerId', 'FirstName', 'LastName', 'Company', 'Address', 'City',
       'State', 'Country', 'PostalCode', 'Phone', 'Fax', 'Email',
       'SupportRepId'],
      dtype='object')

employees columns:  Index(['EmployeeId', 'LastName', 'FirstName', 'Title', 'ReportsTo',
       'BirthDate', 'HireDate', 'Address', 'City', 'State', 'Country',
       'PostalCode', 'Phone', 'Fax', 'Email'],
      dtype='object')


In [72]:
dataframe = (
    employees
    .merge(customers, left_on='EmployeeId', right_on='SupportRepId', 
           how = 'left', suffixes=['_emp','_cust'])
    .groupby(['EmployeeId','LastName_emp', 'FirstName_emp', 'Title'], 
             as_index=False, dropna=False)['CustomerId']
    .count()
    .rename(columns = {'CustomerId':'Num_Customers'})
)
dataframe['Employee_Name'] = (
    dataframe['FirstName_emp'] + ' ' + dataframe['LastName_emp']
)
dataframe = dataframe.drop(['FirstName_emp', 'LastName_emp'], axis=1)
dataframe = dataframe[['EmployeeId', 'Employee_Name', 'Title', 'Num_Customers']]

print(dataframe.to_string(index=False))

 EmployeeId    Employee_Name               Title  Num_Customers
          1     Andrew Adams     General Manager              0
          2    Nancy Edwards       Sales Manager              0
          3     Jane Peacock Sales Support Agent             21
          4    Margaret Park Sales Support Agent             20
          5    Steve Johnson Sales Support Agent             18
          6 Michael Mitchell          IT Manager              0
          7      Robert King            IT Staff              0
          8   Laura Callahan            IT Staff              0
